In [ ]:
config = {
    "codebeamer": {
        "base-url": (BASE_URL := "https://codebeamer.ssi.dbmc.comp.db.de/api/v3/"),
        "auth": {"username": "USER", "password": "PASSWORD"},
    },
    "trackers": [
        {
            "external-id": "5183",
            "capella-uuid": "3be8d0fc-c693-4b9b-8fa1-d59a9eec6ea4",
            "fields": [
                "Capella ID",
                "Business Value",
                "Type",
                "Status",
                "Color",
                "Submitted at",
                "Submitted by",
            ],
        }
    ],
    # Only needed for ChangeSet export & application
    "model": { "path": "model/RM Bridge Show.aird" }
}

### (Optional) Personal proxy configurations

In [ ]:
import requests

socks = "socks5h://localhost:23161"
session = requests.Session()
session.proxies = {"http": socks, "https": socks}
auth = config["codebeamer"]["auth"]
session.auth = (auth["username"], auth["password"])

### Export a Snapshot from RM Tool (CodebeamerAPI)

In [ ]:
import pathlib
import yaml

from rm_bridge_codebeamer_importer import codebeamer, export_tracker_snapshots

api = codebeamer.CodebeamerAPI.from_config({"base_url": BASE_URL, "session": session})
snapshot = export_tracker_snapshots(api, config)
pathlib.Path("test_snapshots.yaml").write_text(
    yaml.dump(snapshot), encoding="utf-8"
)

### Export a ChangeSet (Actions) to synchronize ReqIFElements of a Capella model

In [ ]:
import capellambse
import pathlib
import yaml

from rm_bridge.model_modifier import changeset

model = capellambse.MelodyModel(**config["model"])
change = changeset.calculate_change_set(model, config, snapshot)
pathlib.Path("test_changeset.yaml").write_text(
    yaml.dump(change), encoding="utf-8"
)

### Apply `[Create|Mod|Delete]`Actions to the Capella model

In [ ]:
from rm_bridge.model_modifier import apply_change

tracker = config["trackers"][0]
model_change = apply_change.ModelChange(model, changeset.find.ReqFinder(model))
model_change.apply_changes(
    tracker["external-id"],
    tracker["capella-uuid"],
    change[tracker["external-id"]],
)

### Write changes to model files

In [ ]:
model.save()

In [ ]:
model.la.requirement_modules[0].folders[0]

In [ ]:
model.la.requirement_modules[0].folders[-1]

In [ ]:
model.by_uuid("")